# Testing des Models in Echtzeit

Version updated: 12.08.2019 (latest version)

ToDO:

Colorize the Segments before appending to scene -->Done

project points to camera

In [ ]:
from pyntcloud import PyntCloud
import os
import numpy as np
from random import randint, uniform
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras.callbacks import TensorBoard
import math
import time
import cv2
from sklearn.cluster import KMeans
import open3d as o3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Visualize the test data
import scipy.io as io
import scipy.ndimage as nd
from scipy.spatial import cKDTree
import pyrealsense2 as rs

# Restore the trained model
model = tf.keras.models.load_model('savedmodel/k_model_Washington.h5')

# Variables
disparity_shift = 96


# ||||||||||||||||||||||||||||||||||||||||||||||||||||  Functions ||||||||||||||||||||||||||||||||||||||||||||||||||||


class AppState:

    def __init__(self, *args, **kwargs):
        self.WIN_NAME = 'RealSense'
        self.pitch, self.yaw = math.radians(-10), math.radians(-15)
        self.translation = np.array([0, 0, -1], dtype=np.float32)
        self.distance = 2
        self.prev_mouse = 0, 0
        self.mouse_btns = [False, False, False]
        self.paused = False
        self.decimate = 0
        self.scale = True
        self.color = True
        self.spatial = False
        self.holefilling = False
        self.segmentation = False
        self.minthreshold = 0.2
        self.maxthreshold = 0.8
        self.threshold = False
        

    def reset(self):
        self.pitch, self.yaw, self.distance = 0, 0, 2
        self.translation[:] = 0, 0, -1

    @property
    def rotation(self):
        Rx, _ = cv2.Rodrigues((self.pitch, 0, 0))
        Ry, _ = cv2.Rodrigues((0, self.yaw, 0))
        return np.dot(Ry, Rx).astype(np.float32)

    @property
    def pivot(self):
        return self.translation + np.array((0, 0, self.distance), dtype=np.float32)
    

def mouse_cb(event, x, y, flags, param):

    if event == cv2.EVENT_LBUTTONDOWN:
        state.mouse_btns[0] = True

    if event == cv2.EVENT_LBUTTONUP:
        state.mouse_btns[0] = False

    if event == cv2.EVENT_RBUTTONDOWN:
        state.mouse_btns[1] = True

    if event == cv2.EVENT_RBUTTONUP:
        state.mouse_btns[1] = False

    if event == cv2.EVENT_MBUTTONDOWN:
        state.mouse_btns[2] = True

    if event == cv2.EVENT_MBUTTONUP:
        state.mouse_btns[2] = False

    if event == cv2.EVENT_MOUSEMOVE:

        h, w = out.shape[:2]
        dx, dy = x - state.prev_mouse[0], y - state.prev_mouse[1]

        if state.mouse_btns[0]:
            state.yaw += float(dx) / w * 2
            state.pitch -= float(dy) / h * 2

        elif state.mouse_btns[1]:
            dp = np.array((dx / w, dy / h, 0), dtype=np.float32)
            state.translation -= np.dot(state.rotation, dp)

        elif state.mouse_btns[2]:
            dz = math.sqrt(dx**2 + dy**2) * math.copysign(0.01, -dy)
            state.translation[2] += dz
            state.distance -= dz

    if event == cv2.EVENT_MOUSEWHEEL:
        dz = math.copysign(0.1, flags)
        state.translation[2] += dz
        state.distance -= dz

    state.prev_mouse = (x, y)
    
    

def project(v):
    """project 3d vector array to 2d"""
    h, w = out.shape[:2]
    view_aspect = float(h)/w

    # ignore divide by zero for invalid depth
    with np.errstate(divide='ignore', invalid='ignore'):
        proj = v[:, :-1] / v[:, -1, np.newaxis] * \
            (w*view_aspect, h) + (w/2.0, h/2.0)

    # near clipping
    znear = 0.03
    proj[v[:, 2] < znear] = np.nan
    return proj



def view(v):
    """apply view transformation on vector array"""
    return np.dot(v - state.pivot, state.rotation) + state.pivot - state.translation



def line3d(out, pt1, pt2, color=(0x80, 0x80, 0x80), thickness=1):
    """draw a 3d line from pt1 to pt2"""
    p0 = project(pt1.reshape(-1, 3))[0]
    p1 = project(pt2.reshape(-1, 3))[0]
    if np.isnan(p0).any() or np.isnan(p1).any():
        return
    p0 = tuple(p0.astype(int))
    p1 = tuple(p1.astype(int))
    rect = (0, 0, out.shape[1], out.shape[0])
    inside, p0, p1 = cv2.clipLine(rect, p0, p1)
    if inside:
        cv2.line(out, p0, p1, color, thickness, cv2.LINE_AA)

        

def grid(out, pos, rotation=np.eye(3), size=1, n=10, color=(0x80, 0x80, 0x80)):
    """draw a grid on xz plane"""
    pos = np.array(pos)
    s = size / float(n)
    s2 = 0.5 * size
    for i in range(0, n+1):
        x = -s2 + i*s
        line3d(out, view(pos + np.dot((x, 0, -s2), rotation)),
               view(pos + np.dot((x, 0, s2), rotation)), color)
    for i in range(0, n+1):
        z = -s2 + i*s
        line3d(out, view(pos + np.dot((-s2, 0, z), rotation)),
               view(pos + np.dot((s2, 0, z), rotation)), color)


        
def axes(out, pos, rotation=np.eye(3), size=0.075, thickness=2):
    """draw 3d axes"""
    line3d(out, pos, pos +
           np.dot((0, 0, size), rotation), (0xff, 0, 0), thickness)
    line3d(out, pos, pos +
           np.dot((0, size, 0), rotation), (0, 0xff, 0), thickness)
    line3d(out, pos, pos +
           np.dot((size, 0, 0), rotation), (0, 0, 0xff), thickness)


    
    
def frustum(out, intrinsics, color=(0x40, 0x40, 0x40)):
    """draw camera's frustum"""
    orig = view([0, 0, 0])
    w, h = intrinsics.width, intrinsics.height

    for d in range(1, 6, 2):
        def get_point(x, y):
            p = rs.rs2_deproject_pixel_to_point(intrinsics, [x, y], d)
            line3d(out, orig, view(p), color)
            return p

        top_left = get_point(0, 0)
        top_right = get_point(w, 0)
        bottom_right = get_point(w, h)
        bottom_left = get_point(0, h)

        line3d(out, view(top_left), view(top_right), color)
        line3d(out, view(top_right), view(bottom_right), color)
        line3d(out, view(bottom_right), view(bottom_left), color)
        line3d(out, view(bottom_left), view(top_left), color)

        
        

def pointcloud(out, verts, texcoords, color, painter=True):
    """draw point cloud with optional painter's algorithm"""
    if painter:
        # Painter's algo, sort points from back to front

        # get reverse sorted indices by z (in view-space)
        # https://gist.github.com/stevenvo/e3dad127598842459b68
        v = view(verts)
        s = v[:, 2].argsort()[::-1]
        proj = project(v[s])
    else:
        proj = project(view(verts))

    if state.scale:
        proj *= 0.5**state.decimate

    h, w = out.shape[:2]

    # proj now contains 2d image coordinates
    j, i = proj.astype(np.uint32).T

    # create a mask to ignore out-of-bound indices
    im = (i >= 0) & (i < h)
    jm = (j >= 0) & (j < w)
    m = im & jm

    cw, ch = color.shape[:2][::-1]
    if painter:
        # sort texcoord with same indices as above
        # texcoords are [0..1] and relative to top-left pixel corner,
        # multiply by size and add 0.5 to center
        v, u = (texcoords[s] * (cw, ch) + 0.5).astype(np.uint32).T
    else:
        v, u = (texcoords * (cw, ch) + 0.5).astype(np.uint32).T
    # clip texcoords to image
    np.clip(u, 0, ch-1, out=u)
    np.clip(v, 0, cw-1, out=v)

    # perform uv-mapping
    out[i[m], j[m]] = color[u[m], v[m]]
    

# Colorizes the different Clusters randomly
def colorize_pc(number_clusterpoints):

    rgb = np.zeros((number_clusterpoints, 3), dtype=int)

    r = randint(0, 254)
    g = randint(0, 254)
    b = randint(0, 254)

    rgb_temp = np.array((r,g,b))
    rgb = np.full_like(rgb,rgb_temp)

    return rgb

# Helper Function to display Inliners and Outliners after Outliner Removal

def display_inlier_outlier(cloud, ind):
    inlier_cloud = o3d.geometry.select_down_sample(cloud, ind)
    outlier_cloud = o3d.geometry.select_down_sample(cloud, ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])
                    


# ---------------------------------------------------> Camera Setup <---------------------------------------------------

DS5_product_ids = ["0AD1", "0AD2", "0AD3", "0AD4", "0AD5", "0AF6", "0AFE", "0AFF", "0B00", "0B01", "0B03", "0B07","0B3A"]

def find_device_that_supports_advanced_mode() :
    ctx = rs.context()
    ds5_dev = rs.device()
    devices = ctx.query_devices();
    for dev in devices:
        if dev.supports(rs.camera_info.product_id) and str(dev.get_info(rs.camera_info.product_id)) in DS5_product_ids:
            if dev.supports(rs.camera_info.name):
                print("Found device that supports advanced mode:", dev.get_info(rs.camera_info.name))
            return dev
    raise Exception("No device that supports advanced mode was found")

dev = find_device_that_supports_advanced_mode()
device= rs.context().query_devices()[0]
advnc_mode = rs.rs400_advanced_mode(dev)
depth_table_control_group = advnc_mode.get_depth_table()
depth_table_control_group.disparityShift = disparity_shift
print("Set Disparity Shift to", disparity_shift)
advnc_mode.set_depth_table(depth_table_control_group)


# ---------------------------------------------------> Camera PC Stream <---------------------------------------------------

"""
Mouse: 
    Drag with left button to rotate around pivot (thick small axes), 
    with right button to translate and the wheel to zoom.
Keyboard: 
    [p]     Pause
    [r]     Reset View
    [d]     Cycle through decimation values
    [z]     Toggle point scaling
    [c]     Toggle color source
    [s]     Save PNG (./out.png)
    [e]     Export points to ply (./out.ply)
    [q\ESC] Quit
"""



state = AppState()

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

# Get stream profile and camera intrinsics
profile = pipeline.get_active_profile()
depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
depth_intrinsics = depth_profile.get_intrinsics()
w, h = depth_intrinsics.width, depth_intrinsics.height
pc = rs.pointcloud()


# |||||||||||||||||||||||||||||||||||||||||||||| Filter Setup ||||||||||||||||||||||||||||||||||||||||||||||||||||



decimate = rs.decimation_filter()
decimate.set_option(rs.option.filter_magnitude, 2 ** state.decimate)
colorizer = rs.colorizer()

# Spatial Filter
spatial = rs.spatial_filter()

#hole filling 
hole_filling = rs.hole_filling_filter()

#threshold
threshold_filter = rs.threshold_filter()
threshold_filter.set_option(rs.option.max_distance, state.maxthreshold)
threshold_filter.set_option(rs.option.min_distance, state.minthreshold)


cv2.namedWindow(state.WIN_NAME, cv2.WINDOW_AUTOSIZE)
cv2.resizeWindow(state.WIN_NAME, w, h)
cv2.setMouseCallback(state.WIN_NAME, mouse_cb)

out = np.empty((h, w, 3), dtype=np.uint8)

cluster_scene = np.zeros((0, 6), dtype=float) # Placeholder for the colorized Clusterscene (x,y,z,r,g,b)

while True:
    
    try:
        
        # Grab camera data
        if not state.paused:
            
            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()

            depth_frame = frames.get_depth_frame()
            color_frame = frames.get_color_frame()

            depth_frame = decimate.process(depth_frame)
            depth_frame = threshold_filter.process(depth_frame)


            # Grab new intrinsics (may be changed by decimation)
            depth_intrinsics = rs.video_stream_profile(depth_frame.profile).get_intrinsics()
            w, h = depth_intrinsics.width, depth_intrinsics.height
            
            color_intrinsics = rs.video_stream_profile(color_frame.profile).get_intrinsics()
            
            depth_to_color_extrin = depth_frame.profile.get_extrinsics_to(color_frame.profile)
            color_to_depth_extrin = color_frame.profile.get_extrinsics_to(depth_frame.profile)

            depth_image = np.asanyarray(depth_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())

            depth_colormap = np.asanyarray(colorizer.colorize(depth_frame).get_data())
            
            
# ||||||||||||||||||||||||||||||||||||||||||||||| Filters and more ||||||||||||||||||||||||||||||||||||||||||||||||||

            
            if state.color:
                mapped_frame, color_source = color_frame, color_image
            else:
                mapped_frame, color_source = depth_frame, depth_colormap
            
            if state.holefilling:
                depth_frame = hole_filling.process(depth_frame)
            else:
                depth_frame = frames.get_depth_frame()     
                
            if state.spatial:
                spatial.set_option(rs.option.filter_magnitude, 5)
                spatial.set_option(rs.option.filter_smooth_alpha, 1)
                spatial.set_option(rs.option.filter_smooth_delta, 50)
                spatial.set_option(rs.option.holes_fill, 3)
                depth_frame = spatial.process(depth_frame)
            else:
                depth_frame = frames.get_depth_frame()
           
                
            if state.threshold:
                threshold_filter.set_option(rs.option.max_distance, 0.5)
                threshold_filter.set_option(rs.option.min_distance, 0.25)
                depth_frame = threshold_filter.process(depth_frame)
            else:
                depth_frame = frames.get_depth_frame()
                depth_frame = decimate.process(depth_frame)

            depth_frame = threshold_filter.process(depth_frame)       
                
            points = pc.calculate(depth_frame)
            pc.map_to(mapped_frame)

            # Pointcloud data to arrays
            v, t = points.get_vertices(), points.get_texture_coordinates()
            verts = np.asanyarray(v).view(np.float32).reshape(-1, 3)  # xyz
            texcoords = np.asanyarray(t).view(np.float32).reshape(-1, 2)  # uv
            
            
# |||||||||||||||||||||||||||||||||||||||||||||||||||| SEGMEntATION ||||||||||||||||||||||||||||||||||||||||||||||||||||


            
            if state.segmentation:
                
                state.paused = True
                
                try:
                    
                    x = (verts[:, 0])*(1)
                    y = (verts[:, 1])*(-1) # Flip for correct Camera orientation
                    z = (verts[:, 2])*(-1)
                    
                    df_pointcloud = pd.DataFrame(zip(x, y, z), columns=["x", "y", "z"])
                    pointcloud = PyntCloud(df_pointcloud)
                    
                    initial_pointcloud = PyntCloud(df_pointcloud)
                    
                    #~~~~~~ Outliner Removal ~~~~~~
                    
                    pcd = o3d.geometry.PointCloud()
                    pcd.points = o3d.utility.Vector3dVector(pointcloud.xyz)
                    
                    # Quick Downsamling for better performance
                    uni_down_pcd = o3d.geometry.uniform_down_sample(pcd, every_k_points=50)
                    o3d.visualization.draw_geometries([uni_down_pcd])
                    
                    # the higher std_ration the more agressive is the outliner removal
                    cl, ind = o3d.geometry.statistical_outlier_removal(uni_down_pcd, nb_neighbors=1000, std_ratio=0.8)
                    display_inlier_outlier(uni_down_pcd, ind)
                    
                    #back to numpy array
                    pointcloud = np.asarray(cl.points)
                    
                    
                    x = (pointcloud[:, 0])*(-1) 
                    y = (pointcloud[:, 1])*(-1)
                    z = (pointcloud[:, 2])*(1)
                    
                    df_pointcloud = pd.DataFrame(zip(x, y, z), columns=["x", "y", "z"])
                    pointcloud = PyntCloud(df_pointcloud)
                    
                    pointcloud.plot(initial_point_size=0.005)
                    
                    
                    # Additional Filtering
                    
                    #RANSAC plane fitting
                    #surface = pointcloud.add_scalar_field("plane_fit",n_inliers_to_stop=len(pointcloud.points))
                    #pointcloud.plot(use_as_color=surface, cmap="RdYlGn", output_name="surface")
                    # creates a boolean array
                    #not_surface = pointcloud.points[surface] != 1
                    #pointcloud.apply_filter(not_surface)
                    
                    voxelgrid_id = pointcloud.add_structure("voxelgrid", n_x=128, n_y=128, n_z=128)
                    voxelscene = pointcloud.structures[voxelgrid_id]
                    

                    #clusters_id = pointcloud.add_scalar_field("euclidean_clusters", voxelgrid=voxelgrid_id)
                    #pointcloud.plot(use_as_color=clusters_id, cmap="cool")
                    
                    
                    # Create binary array from Voxelscene
                    binary_voxelscene = voxelscene.get_feature_vector(mode="binary")

                    # Prepare data for Network input
                    binary_voxelscene = np.expand_dims(binary_voxelscene, axis=0)
                    print(binary_voxelscene.shape)

                    # Classification Output on Testvoxels
                    cnn_out = model.predict(binary_voxelscene)

                    cnn_prediction = np.argmax(cnn_out)

                    print("Predicted Numbers of Objects:", cnn_prediction)
                    
                    # Clustering
                    kmeans = KMeans(n_clusters=cnn_prediction, init='k-means++')

                    pointcloud = pointcloud.xyz

                    kmeans = kmeans.fit(pointcloud)
                    labels = kmeans.predict(pointcloud)
                    centroids = kmeans.cluster_centers_

                    # Create Clusterscene

                    clusters_dict = {i: pointcloud[np.where(labels == i)[0]] for i in range(kmeans.n_clusters)}

                    # COLORIZE the points

                    i = 0
                    for i in range(kmeans.n_clusters):
                        cluster = clusters_dict[i]
                        number_clusterpoints = cluster.shape[0]
                        colorize_rgb = colorize_pc(number_clusterpoints)

                        cluster = np.hstack((cluster, colorize_rgb))
                        cluster_scene = np.append(cluster_scene, cluster, axis=0)

                    x = cluster_scene[:, 0]
                    y = cluster_scene[:, 1]
                    z = cluster_scene[:, 2]

                    red = cluster_scene[:, 3]
                    blue = cluster_scene[:, 4]
                    green = cluster_scene[:, 5]

                    df_cluster_scene = pd.DataFrame(zip(x, y, z, red, green, blue), columns=["x", "y", "z",
                                                                                             "red", "green", "blue"])

                    cluster_scene = PyntCloud(df_cluster_scene)
                    print(cluster_scene)
                    scene = cluster_scene.plot(initial_point_size=0.001)
                    
                    print("Cluster Plotting done. Continuing")
                    
                    # map Pointcloud to Cameragrid
                    i=0
                    """
                    while i < cluster_scene.shape[0]:
                        
                    
                        depth_point = [cluster_scene.points.x[i] , cluster_scene.points.y[i],cluster_scene.points.z[i]]
                        color_pixel[i] = rs.rs2_project_point_to_pixel(color_intrinsics, depth_point)
                        print("color pixel", color_pixel)
                    """
                    state.paused = False
                    
                    



                except:
                    print("error")
                    cv2.destroyAllWindows()
                    break
                    pipeline.stop()

                    
                
                

        # Render
        now = time.time()

        out.fill(0)

        grid(out, (0, 0.5, 1), size=1, n=10)
        frustum(out, depth_intrinsics)
        axes(out, view([0, 0, 0]), state.rotation, size=0.1, thickness=1)

        if not state.scale or out.shape[:2] == (h, w):
            pointcloud(out, verts, texcoords, color_source)
        else:
            tmp = np.zeros((h, w, 3), dtype=np.uint8)
            pointcloud(tmp, verts, texcoords, color_source)
            tmp = cv2.resize(
                tmp, out.shape[:2][::-1], interpolation=cv2.INTER_NEAREST)
            np.putmask(out, tmp > 0, tmp)

        if any(state.mouse_btns):
            axes(out, view(state.pivot), state.rotation, thickness=4)

        dt = time.time() - now

        cv2.setWindowTitle(
            state.WIN_NAME, "RealSense (%dx%d) %dFPS (%.2fms) %s" %
            (w, h, 1.0/dt, dt*1000, "PAUSED" if state.paused else ""))
        
        #  ----------------------------------- cv2 Output -----------------------------------------------
        #if state.segmentation:
        #    segmentationstream = np.hstack((out, pointcloud))
        #    cv2.imshow(state.WIN_NAME, segmentationstream)
        #    key = cv2.waitKey(1)
            
        #elsee
        
        colorstream = np.hstack((out, color_image))
        
        cv2.imshow(state.WIN_NAME, colorstream)
        key = cv2.waitKey(1)

        if key == ord("x"): # Segmentation
            state.segmentation ^= True
            
        if key == ord("r"):
            state.reset()

        if key == ord("p"):
            state.paused ^= True

        if key == ord("d"):
            state.decimate = (state.decimate + 1) % 3
            decimate.set_option(rs.option.filter_magnitude, 2 ** state.decimate)
            print("decimation at", state.decimate)
 
        if key == ord("4"):
            state.minthreshold = (state.minthreshold + 0.1)
            threshold_filter.set_option(rs.option.min_distance, state.minthreshold)
            print("min threshold at", state.minthreshold)
            
        if key == ord("5"):
            state.maxthreshold = (state.maxthreshold - 0.1)
            threshold_filter.set_option(rs.option.max_distance, state.maxthreshold)
            print("max threshold at", state.maxthreshold)

        
        if key == ord("3"):
            state.threshold ^= True
            
        if key == ord("1"):            
            state.spatial ^= True
            
        if key == ord("2"):
            state.holefilling ^= True
            
        if key == ord("z"):
            state.scale ^= True

        if key == ord("c"):
            state.color ^= True

        if key == ord("s"):
            cv2.imwrite('./out.png', out)

        if key == ord("e"):
            points.export_to_ply('./out.ply', mapped_frame)

        if key in (27, ord("q")) or cv2.getWindowProperty(state.WIN_NAME, cv2.WND_PROP_AUTOSIZE) < 0:
            cv2.destroyAllWindows()
            break
    except:
        break
        cv2.destroyAllWindows()
        print("An Error occured. Stopping everything")
        pipeline.stop()
            
# Stop streaming
pipeline.stop()


In [ ]:
#voxelscene.plot(d=3, mode="density", cmap="hsv")





In [ ]:
# Pointcloud to 2D Camera Grid

pixel_clusterscene = np.empty((0,2))

i=0
k=0
while i < cluster_scene.xyz.shape[0]:


    depth_point = [cluster_scene.points.x[i], cluster_scene.points.y[i],cluster_scene.points.z[i]]

    color_point = rs.rs2_transform_point_to_point(depth_to_color_extrin, depth_point)
    color_pixel = rs.rs2_project_point_to_pixel(color_intrinsics, color_point)
    
    pixel_clusterscene = np.vstack((pixel_clusterscene, color_pixel))
    
    i = i + 1

pixel_clusterscene = np.around(pixel_clusterscene)
pixel_clusterscene = pixel_clusterscene.astype(int)
print("color pixel", pixel_clusterscene.shape)


x_values = np.array((1,1), dtype = int)
y_values = np.array((1,1), dtype = int)

i=0
for points in pixel_clusterscene:
    x_values = np.concatenate((x_values, points[0]), axis=None)

    y_values = np.concatenate((y_values, points[1]), axis=None)
    i = i + 1


x_max = np.max(x_values) +1
y_max = np.max(y_values) +1
print(x_max,y_max)


# Mapping 
final_clusterscene = np.zeros((x_max, y_max), dtype = int)

for points in pixel_clusterscene:
    final_clusterscene[points[0]][points[1]] = 255

np.save("pixel_clusterscene",final_clusterscene)


#cv2.imshow("test", final_clusterscene) 
#key = cv2.waitKey(1)


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


a = np.load("pixel_clusterscene.npy")

print(a.shape)


plt.imshow(a)

Testfall aufbauen:

3D CNN gegen 2D CNN stärken und schwächen. Verbinden mit Robot Vision oder Augmented Reality

Verschiedene Belichtungen 

Verdeckungen



Limitationen:
Datensatz
Parameter Netzwerk



.object file erstellen von jedem segment ->  Import to Unity3dVR

ziel:

automatsche segmentation für scenen in VR

besser als händische segmentation in unity z.b


warum besser als sematic segmentation

In [ ]:
cv2.imshow("test", a) 
key = cv2.waitKey(1)

In [ ]:

# Load RealSense .PLY Data to PyntCloud Object
pointcloud = PyntCloud.from_file("out.ply") 
#pointcloud = pointcloud.get_sample("mesh_random", n=50000, rgb=True, normals=False, as_PyntCloud=True)

In [ ]:
scene = pointcloud.plot(initial_point_size=0.001)

In [ ]:

# ------------------------------------------ Voxilisation -----------------------------------------------------

# create Voxelgrid from Pointcloudscene
voxelgrid_id = pointcloud.add_structure("voxelgrid", n_x=128, n_y=128, n_z=128)
voxelscene = pointcloud.structures[voxelgrid_id]

# Create binary array from Voxelscene 
binary_voxelscene = voxelscene.get_feature_vector(mode="binary")

voxelscene.plot(d=3, mode="density", cmap="hsv")


In [ ]:
# Prepare data for Network input
binary_voxelscene = np.expand_dims(binary_voxelscene, axis=0)
print(binary_voxelscene.shape)

# Classification Output on Testvoxels
cnn_out = model.predict(binary_voxelscene)

cnn_prediction = np.argmax(cnn_out)

print("Predicted Numbers of Objects:", cnn_prediction)

In [ ]:
# ----------------------------------------------- K Means Clustering --------------------------------------------------

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = cnn_prediction, init='k-means++')

#pointcloud = pointcloud.xyz

kmeans = kmeans.fit(pointcloud)
labels = kmeans.predict(pointcloud)
centroids = kmeans.cluster_centers_

fig = plt.figure()
ax = Axes3D(fig)
pe = ax.scatter(pointcloud[:, 0], pointcloud[:, 1], pointcloud[:, 2], c=labels, cmap='viridis',
            edgecolor='k', s=3, alpha = 0.3)

In [ ]:
print(labels.size)

In [ ]:
# ----------------------------------- Evaluation of Clustering ----------------------------------------------------

clusters_dict = {i: pointcloud[np.where(labels == i)[0]] for i in range(kmeans.n_clusters)}

#  dictionary to list 
clusters_list = []
for key, value in clusters_dict.items():
    temp = [key,value]
    clusters_list.append(temp)
    
# Plotten der einzelnen Cluster    
for i in range(kmeans.n_clusters):
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.scatter(clusters_dict[i][:,0], clusters_dict[i][:, 1], clusters_dict[i][:, 2], 
               cmap='viridis',edgecolor='k', s=1, alpha = 0.1)

In [ ]:
# ---------------------  Verteilung der Punkte/Cluster zu Punkte/Original PC -------------------------------------------

c = 0   # Counter for Clusters
p = 0   # Counter for Pointclouds
counter = 0     # Counter for matches between Pointclouds and Clusters
allocation_list = np.zeros((3, 3), dtype=int)

''' 
Structure of allocation list

    p0  p1  p2 ...
c0
c1 
c2
...
'''

while c < 3:

    for Pj in clusters_dict[c]:
        for Pi in labeled_pointcloudscene[p]:
            if np.all(Pj == Pi):
                counter = counter + 1

    allocation_list[c, p] = counter
    p = p + 1
    counter = 0

    if p > cnn_prediction:  # max anzahl an pointclouds rounded_cnn
        c = c + 1
        p = 0
    else:
       continue

print(allocation_list)

In [ ]:
allocation_max = np.amax(allocation_list, axis=1)
x = 0
y = 0
allocation_result={}

'''
    returns allocation between Clusters/ PClouds like: (Clusternumber,Pointcloudnumber), ...
    --> Prediction based of where is the max Value 
'''

while y <= (len(allocation_max) - 1):  # len q = 3 but we start at 0 
    
    result = (np.where(allocation_list[y] == allocation_max[y]))
    allocation_result[y] =  y , result[0][0]
    print("Cluster",y,"belongs to", "Pointcloud",result[0][0])
    y = y + 1 

allocation_result

In [ ]:
i = 0 
score = {}
temp = 0
max_value = np.amax(allocation_list)

# Calculates for every existing Cluster the score
while i <= 2:
    for d in allocation_list[i]:  
        if (d > 0) and (d < allocation_max[i]):  # d > 0 < clusters maxvalue
            temp = 0
            temp = temp + d
        elif (d > 0) and (d < max_value): # d > 0 < overall max value in allocation list
            temp = 0
            temp = temp + (max_value - d)
        score[i] = temp
    print("Cluster",i, "score=", score[i])

    i = i + 1
    
score